In [ ]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
#from simple_salesforce import Salesforce
#import shutil
#import datetime
#from datetime import date
import json, re
#from pathlib import Path
#import pandas as pd
#import numpy as np
#from io import StringIO
import shutil
import datetime
from datetime import datetime,date
#import re
#import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()


""" sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm01041983*",
    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",
) """

In [ ]:
with open("../login.json", "r") as j:
    misDatos = json.load(j)

In [ ]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [95]:
# Clinica_deporte

query_deportes = "SELECT Id,Name,CreatedDate,Canal__c, RecordTypeId, Seleccione__c, Seleccione2__c,  Seleccione4__c,Realizado__c,Tipo_de_Atenci_n__c from Ficha_Cl_nica_Deporte__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 ORDER BY CreatedDate"
df_deportes = sf.smart_query(query_deportes)

df_deportes["Seleccione__c"] = df_deportes["Seleccione__c"].astype(str).replace("Se intenta contactar para realizar ingreso, sin éxito.", "1")
df_deportes["Seleccione2__c"] = df_deportes["Seleccione2__c"].astype(str).replace("Se intenta contactar una vez para realizar seguimiento, sin éxito.", "1")
df_deportes["Seleccione2__c"] = df_deportes["Seleccione2__c"].astype(str).replace("Se intenta contactar dos veces para realizar seguimiento, sin éxito.", "1")
df_deportes["Seleccione4__c"] = df_deportes["Seleccione4__c"].astype(str).replace("No Contesta", "1")


df_deportes["Realizado"] = np.where((df_deportes["Seleccione__c"] != "1") & (df_deportes["Seleccione2__c"]!= "1") & (df_deportes["Seleccione4__c"]!="1"),"SI","NO")
df_deportes["Realizado"] = np.where((df_deportes["RecordTypeId"] == "0128c000002wQ4hAAE"),"NO",df_deportes["Realizado"])
df_deportes["Realizado"] = np.where((df_deportes["RecordTypeId"] == "0128c000002wQ4mAAE"),"NO",df_deportes["Realizado"])
df_deportes["Realizado"] = np.where((df_deportes["RecordTypeId"] == "0128c000002CYp6AAG"),"NO",df_deportes["Realizado"])
condlist = [
    df_deportes["Canal__c"] == "Chatter",
    df_deportes["Canal__c"] == "Llamada",
    df_deportes["Canal__c"] == "Videollamada",
    df_deportes["Canal__c"] == "Correo electrónico",
]
choicelist = [
    "Derivación",
    "Atención Agendada",
    "Atención Agendada",
    "Complemento a la atención",
]
df_deportes["Tipo"] = np.select(condlist, choicelist)



df_deportes["Clínica"] = "Deporte"

df_deportes.head()


100%|██████████| 1/1 [00:08<00:00,  8.25s/it]


,Id,Name,CreatedDate,Canal__c,RecordTypeId,Seleccione__c,Seleccione2__c,Seleccione4__c,Realizado__c,Tipo_de_Atenci_n__c,Realizado,Tipo,Clínica
0,a1bRN000000WmCnYAK,FCD-7118,1704197510000,Llamada,0128c000002CYpBAAW,-,-,-,SI,Atención agendada,SI,Atención Agendada,Deporte
1,a1bRN000000WmEPYA0,FCD-7119,1704197931000,Llamada,0128c000002CYp4AAG,-,-,-,SI,Atención agendada,SI,Atención Agendada,Deporte
2,a1bRN000000WmfpYAC,FCD-7120,1704199491000,Llamada,0128c000002CYp9AAG,-,-,-,SI,Atención agendada,SI,Atención Agendada,Deporte
3,a1bRN000000WmpVYAS,FCD-7121,1704201197000,Llamada,0128c000002CYp6AAG,-,-,-,NO,Atención agendada,NO,Atención Agendada,Deporte
4,a1bRN000000Wmr7YAC,FCD-7122,1704201852000,Llamada,0128c000002CYp8AAG,-,-,Se Realiza,SI,Atención agendada,SI,Atención Agendada,Deporte


In [97]:
#Clínica Nutrición
query_nutricion = "SELECT Id,Name,CreatedDate,Canal__c, RecordTypeId,Seleccione5__c,Realizado__c,Tipo_de_Atenci_n__c from Ficha_Clinica_Nutrici_n__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 ORDER BY CreatedDate"
df_nutricion = sf.smart_query(query_nutricion)
df_nutricion["Seleccione5__c"] = (
    df_nutricion["Seleccione5__c"].astype(str).replace("No Contesta", "1")
)
df_nutricion["Realizado"] = np.where(
    (df_nutricion["Seleccione5__c"] != "1"), "SI", "NO"
)
df_nutricion["Realizado"] = np.where((df_nutricion["RecordTypeId"] == "0128c000002wQ2gAAE"),"NO",df_nutricion["Realizado"])
df_nutricion["Realizado"] = np.where((df_nutricion["RecordTypeId"] == "0128c000002wQ3yAAE"),"NO",df_nutricion["Realizado"])
df_nutricion["Realizado"] = np.where((df_nutricion["RecordTypeId"] == "0128c000002wOXWAA2"),"NO",df_nutricion["Realizado"])

condlist = [
    df_nutricion["Canal__c"] == "Chatter",
    df_nutricion["Canal__c"] == "Llamada",
    df_nutricion["Canal__c"] == "Videollamada",
    df_nutricion["Canal__c"] == "Correo electrónico",
]
choicelist = [
    "Derivación",
    "Atención Agendada",
    "Atención Agendada",
    "Complemento a la atención",
]
df_nutricion["Tipo"] = np.select(condlist, choicelist)
df_nutricion["Clínica"] = "Nutrición"


df_nutricion.head()

100%|██████████| 1/1 [00:08<00:00,  8.51s/it]


,Id,Name,CreatedDate,Canal__c,RecordTypeId,Seleccione5__c,Realizado__c,Tipo_de_Atenci_n__c,Realizado,Tipo,Clínica
0,a1XRN000000DgMP2A0,FCN-0016441,1704199015000,Llamada,0128c000002wQ3yAAE,-,NO,Atención agendada,NO,Atención Agendada,Nutrición
1,a1XRN000000DgO12AK,FCN-0016442,1704201719000,Videollamada,0128c000002wOXVAA2,-,SI,Atención agendada,SI,Atención Agendada,Nutrición
2,a1XRN000000DgRF2A0,FCN-0016443,1704201990000,Llamada,0128c000002wOXOAA2,1,NO,Atención agendada,NO,Atención Agendada,Nutrición
3,a1XRN000000DgW52AK,FCN-0016444,1704203342000,Videollamada,0128c000002wOXSAA2,-,SI,Atención agendada,SI,Atención Agendada,Nutrición
4,a1XRN000000DgZJ2A0,FCN-0016445,1704203620000,Llamada,0128c000002wOXSAA2,-,SI,Atención agendada,SI,Atención Agendada,Nutrición


In [ ]:
#Psicologia
query_psicologia = "SELECT Id,Name,CreatedDate,Canal__c, RecordTypeId, Motivo_de_consulta_manifiesto__c from Ficha_Cl_nica_Asistencia_Emocional__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 ORDER BY CreatedDate"
df_psicologia = sf.smart_query(query_psicologia)


df_psicologia["Realizado"] = "SI"
df_psicologia["Realizado"] = np.where((df_psicologia["RecordTypeId"] == "0128c000002wQ51AAE"),"NO",df_psicologia["Realizado"])
df_psicologia["Realizado"] = np.where((df_psicologia["RecordTypeId"] == "0128c000002wQ56AAE"),"NO",df_psicologia["Realizado"])
df_psicologia["Realizado"] = np.where((df_psicologia["RecordTypeId"] == "0128c000002CYoyAAG"),"NO",df_psicologia["Realizado"])

df_psicologia["Motivo_de_consulta_manifiesto__c"] = df_psicologia["Motivo_de_consulta_manifiesto__c"].str.replace("-", "Sin Registro")

condlist = [
    df_psicologia["Canal__c"] == "Chatter",
    df_psicologia["Canal__c"] == "llamada",
    df_psicologia["Canal__c"] == "Videollamada",
    df_psicologia["Canal__c"] == "Correo electrónico",
]
choicelist = [
    "Derivación",
    "Atención Agendada",
    "Atención Agendada",
    "Complemento a la atención",
]
df_psicologia["Tipo"] = np.select(condlist, choicelist)
df_psicologia["Clínica"] = "Nutrición"


df_psicologia.head()

In [ ]:

query_general = "SELECT Id,Name,CreatedDate,Canal__c,Programa__c from Ficha_Clinica_General__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 ORDER BY CreatedDate"
df_general = sf.smart_query(query_general)

df_general["Realizado"] = "SI"



df_general.head()

In [81]:

query_sueno = "SELECT Id,Name,CreatedDate,Canal__c,RecordTypeId,Realizado__c,Tipo_de_Atenci_n__c from Ficha_Cl_nica_del_Sue_o__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 ORDER BY CreatedDate"
df_sueno = sf.smart_query(query_sueno)

df_sueno["Realizado"] = "SI"

df_sueno["Realizado"] = np.where((df_sueno["RecordTypeId"] == "0128c000002CYq4AAG"),"NO",df_sueno["Realizado"])
df_sueno["Realizado"] = np.where((df_sueno["RecordTypeId"] == "0128c000002wQ5pAAE"),"NO",df_sueno["Realizado"])
df_sueno["Realizado"] = np.where((df_sueno["RecordTypeId"] == "0128c000002wQ5uAAE"),"NO",df_sueno["Realizado"])





100%|██████████| 1/1 [00:02<00:00,  2.64s/it]


In [98]:
df_nutricion.head(50)

,Id,Name,CreatedDate,Canal__c,RecordTypeId,Seleccione5__c,Realizado__c,Tipo_de_Atenci_n__c,Realizado,Tipo,Clínica
0,a1XRN000000DgMP2A0,FCN-0016441,1704199015000,Llamada,0128c000002wQ3yAAE,-,NO,Atención agendada,NO,Atención Agendada,Nutrición
1,a1XRN000000DgO12AK,FCN-0016442,1704201719000,Videollamada,0128c000002wOXVAA2,-,SI,Atención agendada,SI,Atención Agendada,Nutrición
2,a1XRN000000DgRF2A0,FCN-0016443,1704201990000,Llamada,0128c000002wOXOAA2,1,NO,Atención agendada,NO,Atención Agendada,Nutrición
3,a1XRN000000DgW52AK,FCN-0016444,1704203342000,Videollamada,0128c000002wOXSAA2,-,SI,Atención agendada,SI,Atención Agendada,Nutrición
4,a1XRN000000DgZJ2A0,FCN-0016445,1704203620000,Llamada,0128c000002wOXSAA2,-,SI,Atención agendada,SI,Atención Agendada,Nutrición
5,a1XRN000000Dgav2AC,FCN-0016446,1704203792000,Llamada,0128c000002wOXRAA2,-,SI,Atención agendada,SI,Atención Agendada,Nutrición
6,a1XRN000000Dgr32AC,FCN-0016447,1704206713000,Llamada,0128c000002wOXUAA2,-,SI,Atención agendada,SI,Atención Agendada,Nutrición
7,a1XRN000000DguH2AS,FCN-0016448,1704207724000,Llamada,0128c000002wOXRAA2,-,SI,Atención agendada,SI,Atención Agendada,Nutrición
8,a1XRN000000Dgvt2AC,FCN-0016449,1704207938000,Videollamada,0128c000002wQ3yAAE,-,NO,Atención agendada,NO,Atención Agendada,Nutrición
9,a1XRN000000Dh3x2AC,FCN-0016450,1704209753000,Videollamada,0128c000002wOXSAA2,-,SI,Atención agendada,SI,Atención Agendada,Nutrición
